Identificar correctamente todas las diferentes variantes de fecha codificadas en este conjunto de datos, normalizar y ordenar adecuadamente las fechas. Cada línea del archivo dates.txt corresponde a una nota médica. Cada nota tiene una fecha que debe extraerse, pero cada fecha está codificada en uno de muchos formatos. 

Aquí hay una lista de algunas de las variantes que puede encontrar en este conjunto de datos:

- 04/20/2009; 04/20/09; 4/20/09; 4/3/09
- Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
- 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
- Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
- Feb 2009; Sep 2009; Oct 2010
- 6/2008; 12/2009
- 2009; 2010
 
Una vez que haya extraído estos patrones de fecha del texto, el siguiente paso es clasificarlos en orden cronológico ascendente de acuerdo con las siguientes reglas:

- Todas las fechas están en formato xx/xx/ xx son mm/dd/aa
- Todas las fechas en las que el año está codificado en solo dos dígitos corresponden a años posteriores a la década de 1900 (p. Ej., 1/5/89 es el 5 de enero de 1989).
- Si falta el día (p. Ej., 9/2009), suponga que es el primer día del mes (p. Ej., septiembre, 1 de 2009).
- Si falta el mes (por ejemplo, 2010), suponga que es el primero de enero de ese año (p. Ej., enero, 1 de 2010).
- Tenga cuidado con los posibles errores tipográficos, ya que este es un conjunto de datos derivados de la vida real.
 
Esta función debería devolver una lista de longitud 500.

In [71]:
import pandas as pd 
# importar el documento dates.txt 
doc = []
with open('dates.txt') as file:
  for line in file:
    doc.append(line)

df = pd.Series(doc)
df.head(20)

0          03/25/93 Total time of visit (in minutes):\n
1                        6/18/85 Primary Care Doctor:\n
2     sshe plans to move as of 7/8/71 In-Home Servic...
3                 7 on 9/27/75 Audit C Score Current:\n
4     2/6/96 sleep studyPain Treatment Pain Level (N...
5                     .Per 7/06/79 Movement D/O note:\n
6     4, 5/18/78 Patient's thoughts about current su...
7     10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                          3/7/86 SOS-10 Total Score:\n
9              (4/10/71)Score-1Audit C Score Current:\n
10    (5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC...
11                        4/09/75 SOS-10 Total Score:\n
12    8/01/98 Communication with referring physician...
13    1/26/72 Communication with referring physician...
14    5/24/1990 CPT Code: 90792: With medical servic...
15    1/25/2011 CPT Code: 90792: With medical servic...
16          4/12/82 Total time of visit (in minutes):\n
17         1; 10/13/1976 Audit C Score, Highest/

In [73]:
import re

fechas = []
for linea in df:
    # Buscar fechas con nombres de mes en español
    match2 = re.findall(r'\d{1,2}/\d{4}', linea)
    if match2:
        fechas.extend(match2)

print(fechas)
len(fechas)

['24/1990', '25/2011', '13/1976', '25/1984', '31/1985', '12/2012', '27/1986', '13/2002', '15/1998', '11/1987', '5/2010', '20/1982', '29/1994', '3/1985', '05/1999', '07/1974', '14/2000', '09/1981', '08/2004', '08/1990', '11/1974', '18/2002', '11/2006', '20/2011', '27/2006', '6/1998', '6/2005', '10/1973', '9/2005', '03/1980', '12/2005', '5/1987', '5/2004', '8/1974', '3/1986', '10/1997', '3/1993', '3/1981', '9/2003', '10/1993', '1/1983', '7/1994', '12/2008', '10/1980', '8/2003', '12/1975', '11/2010', '7/1997', '7/2014', '9/2001', '8/1986', '1/1978', '9/1975', '1/2009', '5/1995', '8/1989', '11/2000', '06/1973', '2/1999', '6/2001', '12/1978', '8/2009', '7/1973', '1/2014', '09/1975', '12/2012', '5/1999', '7/1989', '7/2009', '11/1998', '3/1995', '2/2009', '08/1988', '4/2007', '5/2000', '4/2012', '10/2001', '2/1977', '8/2008', '2/1983', '5/1979', '1/1992', '11/2008', '12/2014', '4/1974', '10/1981', '10/1986', '03/1973', '12/1994', '8/1999', '10/2010', '10/1994', '01/2007', '8/2010', '03/1990',

137

In [72]:
import re
import pandas as pd

lineas = []
fechas_extraidas = []

for linea in df:
    match1 = re.findall(r'(?:\d{1,2} )?[\/\- \s]\d{1,2}[\/\- \s]\d{2,4}', linea)
    match2 = re.findall(r'(?:\d{1,2} )?(?:de )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:de )?(?:\d{2}, )?\d{4}', linea)
    match3 = re.findall(
        r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ .,-]*'
        r'(?:\d{1,2}(?:st|nd|rd|th)?[ ,.-]*)?'
        r'\d{4}', linea)
    fechas = match1 + match2 + match3 
    lineas.append(linea.strip())
    fechas_extraidas.append(', '.join(fechas) if fechas else '')

df_fechas = pd.DataFrame({'linea': lineas, 'fecha_extraida': fechas_extraidas})
df_fechas.head(20)

,linea,fecha_extraida
0,03/25/93 Total time of visit (in minutes):,/25/93
1,6/18/85 Primary Care Doctor:,/18/85
2,sshe plans to move as of 7/8/71 In-Home Servic...,/8/71
3,7 on 9/27/75 Audit C Score Current:,9/27
4,2/6/96 sleep studyPain Treatment Pain Level (N...,/6/96
5,.Per 7/06/79 Movement D/O note:,7/06
6,"4, 5/18/78 Patient's thoughts about current su...",5/18
7,10/24/89 CPT Code: 90801 - Psychiatric Diagnos...,/24/89
8,3/7/86 SOS-10 Total Score:,/7/86
9,(4/10/71)Score-1Audit C Score Current:,/10/71


In [69]:
# Mostrar solo las líneas sin fecha extraída
sin_fecha = df_fechas[df_fechas['fecha_extraida'] == '']
sin_fecha

,linea,fecha_extraida
455,sHemmorage caused by probe in 1984 Medical His...,
456,sHas been at MYH since his treaters in NE reti...,
457,Pt joined Army reserves in 2001 and has 3 year...,
458,one sister from whom he is estranged due to he...,
459,sSince 1998. Prior medication trials (includin...,
460,1 Ex-smoker : quit 2012,
461,". Age 16, 1991, frontal impact. out for two we...",
462,sLexapro (1988-now): Good response (anxiety),
463,s 25 year old engaged to be married Optic...,
464,8Complications from brain hemmorage in 2016 Ax...,


In [39]:
re.findall(r'(?:\d{1,2} )?(?:de )?(?:Ene|Feb|Mar|Abr|May|Jun|Jul|Ago|Sep|Oct|Nov|Dic)[a-z]* (?:de )?(?:\d{2}, )?\d{4}', sin_fecha['linea'])

TypeError: expected string or bytes-like object, got 'Series'